<a href="https://colab.research.google.com/github/Frankl1sales/CLP-EXEMPLOS/blob/main/BERTIMBAUTreinamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
!pip install TensorFlow1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Created wheel for TensorFlow1: filename=TensorFlow1-3.2.0-py3-none-any.whl size=5116 sha256=70d2e3b2b21133cee44e015dec260be490c09a0aaf01cf57027954c5bb371982
  Stored in directory: /root/.cache/pip/wheels/bc/4f/54/4f5183ac8e9d01b48c31b7c654eb0eccaf97a0ee253fc51240
Successfully built TensorFlow1


In [4]:
!pip install flax

In [5]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

In [6]:
!pip show flax

Name: flax
Version: 0.8.1
Summary: Flax: A neural network library for JAX designed for flexibility
Home-page: 
Author: 
Author-email: Flax team <flax-dev@google.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: jax, msgpack, numpy, optax, orbax-checkpoint, PyYAML, rich, tensorstore, typing-extensions
Required-by: dopamine-rl


In [7]:
!pip install transformers

In [1]:
!pip install transformers --upgrade

In [29]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

# Carregue o tokenizador e o modelo de máscara BERT para a linguagem portuguesa
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# Crie o pipeline usando o modelo BERT
pipe = pipeline('fill-mask', model=model, tokenizer=tokenizer)

# Teste preenchendo a máscara em uma sentença
results = pipe('Tinha uma [MASK] no meio do caminho.')
for result in results:
    print(result)

{'score': 0.1428779512643814, 'token': 5028, 'token_str': 'pedra', 'sequence': 'Tinha uma pedra no meio do caminho.'}
{'score': 0.06213385611772537, 'token': 7411, 'token_str': 'árvore', 'sequence': 'Tinha uma árvore no meio do caminho.'}
{'score': 0.05514984950423241, 'token': 5675, 'token_str': 'estrada', 'sequence': 'Tinha uma estrada no meio do caminho.'}
{'score': 0.029918842017650604, 'token': 1105, 'token_str': 'casa', 'sequence': 'Tinha uma casa no meio do caminho.'}
{'score': 0.025660419836640358, 'token': 3466, 'token_str': 'cruz', 'sequence': 'Tinha uma cruz no meio do caminho.'}


#Preparação dos Dados

##Conjunto de Dados de Texto

conjunto de dados que contenha sentenças com palavras mascaradas


In [118]:
# Exemplo de criação de um conjunto de dados fictício
dataset = [
    "O pássaro canta durante a manhã.",
    "O Opelisco está no centro da cidade.",
    "As aranhas são animais fascinantes.",
    "Ela pedala todos os dias.",
    "A paisagem está cheia de estrelas."
]

# Visualizando o conjunto de dados
for sentence in dataset:
    print(sentence)

O pássaro canta durante a manhã.
O Opelisco está no centro da cidade.
As aranhas são animais fascinantes.
Ela pedala todos os dias.
A paisagem está cheia de estrelas.


In [109]:
import random

def mask_words(sentence, mask_prob=0.15):
    # Divide a sentença em palavras
    words = sentence.split()

    # Escolhe aleatoriamente palavras para mascarar
    masked_indices = random.sample(range(len(words)), int(len(words) * mask_prob))

    # Substitui as palavras selecionadas por [MASK]
    for idx in masked_indices:
        words[idx] = "[MASK]"

    # Junta as palavras novamente em uma sentença
    masked_sentence = " ".join(words)

    return masked_sentence


In [54]:
# Criar um novo conjunto de dados com palavras mascaradas
masked_dataset = [mask_words(sentence) for sentence in dataset]

# Visualizando o conjunto de dados mascarado
for sentence in masked_dataset:
    print(sentence)


O [MASK] canta durante a [MASK].
[MASK] está no centro da cidade.
As [MASK] são animais fascinantes.
Ela [MASK] todos os dias.
A [MASK] está cheia de estrelas.


#Pré-processamento dos Dados

## Tokenização das Sentenças
Utilizando a biblioteca **transformers** para tokenizar as setenças

In [55]:
from transformers import AutoTokenizer

# Carregue o tokenizador
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Exemplo de sentenças
sentences = masked_dataset

# Tokenize as sentenças e substitua uma palavra por [MASK]
tokenized_sentences = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
print("Sentenças tokenizadas:", tokenized_sentences)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentenças tokenizadas: {'input_ids': tensor([[  101,   231,   103,  8954,   726,   123,   103,   119,   102,     0],
        [  101,   103,   698,   202,  1997,   180,   651,   119,   102,     0],
        [  101,   510,   103,   453,  3155,  8573,   387,   358,   119,   102],
        [  101,  1660,   103,   944,   259,  1564,   119,   102,     0,     0],
        [  101,   177,   103,   698, 13086,   125,  4425,   119,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


A saída "Sentenças tokenizadas" que você mostrou é um dicionário contendo três tensores:

**input_ids**: IDs dos tokens tokenizados. Cada lista representa uma sentença, onde os IDs dos tokens estão presentes.

**token_type_ids**: IDs que indicam a qual segmento (se é o primeiro ou segundo) cada token pertence.

**attention_mask**: Máscara de atenção indicando quais tokens são reais e quais são preenchidos (0 indica que o token é preenchido, 1 indica um token real).

## Representações Numéricas
Convertendo as setenças tokenizadas em representações numéricas

In [56]:
# Acesse os IDs dos tokens
input_ids = tokenized_sentences["input_ids"]
print(input_ids)

tensor([[  101,   231,   103,  8954,   726,   123,   103,   119,   102,     0],
        [  101,   103,   698,   202,  1997,   180,   651,   119,   102,     0],
        [  101,   510,   103,   453,  3155,  8573,   387,   358,   119,   102],
        [  101,  1660,   103,   944,   259,  1564,   119,   102,     0,     0],
        [  101,   177,   103,   698, 13086,   125,  4425,   119,   102,     0]])


In [57]:
attention_mask = tokenized_sentences['attention_mask']
print(attention_mask)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [58]:
labels = input_ids.clone()
print(labels)

tensor([[  101,   231,   103,  8954,   726,   123,   103,   119,   102,     0],
        [  101,   103,   698,   202,  1997,   180,   651,   119,   102,     0],
        [  101,   510,   103,   453,  3155,  8573,   387,   358,   119,   102],
        [  101,  1660,   103,   944,   259,  1564,   119,   102,     0,     0],
        [  101,   177,   103,   698, 13086,   125,  4425,   119,   102,     0]])


# Carregamento do Modelo

## Carregamento do Modelo BERT Pré-treinado
Usando a biblioteca **transformers** para carregar o modelo **BERTIMBAU**


In [65]:
from transformers import AutoModelForMaskedLM, pipeline

# Carregue o modelo BERTIMBAU pré-treinado
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Teste do Modelo BERTIMBAU

In [67]:
# Crie o pipeline usando o modelo BERT
pipe = pipeline('fill-mask', model=model, tokenizer=tokenizer)

# Teste preenchendo a máscara em uma sentença
results = pipe(masked_dataset)
for result in results:
    print(result)

[[{'score': 0.2047966867685318, 'token': 939, 'token_str': 'grupo', 'sequence': '[CLS] O grupo canta durante a [MASK]. [SEP]'}, {'score': 0.05897710844874382, 'token': 4249, 'token_str': 'cantor', 'sequence': '[CLS] O cantor canta durante a [MASK]. [SEP]'}, {'score': 0.05372759699821472, 'token': 1788, 'token_str': 'público', 'sequence': '[CLS] O público canta durante a [MASK]. [SEP]'}, {'score': 0.033051278442144394, 'token': 4672, 'token_str': 'casal', 'sequence': '[CLS] O casal canta durante a [MASK]. [SEP]'}, {'score': 0.029878173023462296, 'token': 16647, 'token_str': 'coral', 'sequence': '[CLS] O coral canta durante a [MASK]. [SEP]'}], [{'score': 0.0865575522184372, 'token': 2954, 'token_str': 'noite', 'sequence': '[CLS] O [MASK] canta durante a noite. [SEP]'}, {'score': 0.08304905891418457, 'token': 1119, 'token_str': 'música', 'sequence': '[CLS] O [MASK] canta durante a música. [SEP]'}, {'score': 0.08017916977405548, 'token': 4689, 'token_str': 'apresentação', 'sequence': '[CLS

# Treinamento
O objetivo principal ao treinar um modelo BERT para prever palavras mascaradas é capacitá-lo a entender o contexto das palavras em uma frase. Durante o treinamento, algumas palavras nas sentenças de treinamento são mascaradas (substituídas pelo token [MASK]), e o modelo é treinado para prever essas palavras mascaradas com base no contexto fornecido pelas palavras ao redor.

## Treinamento do BERTIMBAU

In [68]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import AutoModelForMaskedLM, AdamW, get_linear_schedule_with_warmup

In [78]:
# Configurar DataLoader
dataset1 = TensorDataset(input_ids, attention_mask, labels)
train_size = int(0.8 * len(dataset1))
val_size = len(dataset1) - train_size
train_dataset, val_dataset = random_split(dataset1, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [81]:
# Configurar o modelo
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 7
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [128]:
def train_model(model, train_dataloader, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0

        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()

            # Descompactar o batch
            input_ids, attention_mask, labels = batch

            # Saídas do modelo
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}')

In [129]:
train_model(model, train_dataloader, optimizer, scheduler, num_epochs)

Epoch 1/7, Average Loss: 2.314837336540222
Epoch 2/7, Average Loss: 2.2978105545043945
Epoch 3/7, Average Loss: 2.287902593612671
Epoch 4/7, Average Loss: 2.3278708457946777
Epoch 5/7, Average Loss: 2.2926270961761475
Epoch 6/7, Average Loss: 2.3373337984085083
Epoch 7/7, Average Loss: 2.304298162460327


# Teste de Modelo
Será utilizado o modelo treinado para prever novas palavras

In [85]:
import requests
from bs4 import BeautifulSoup

In [98]:
url = "https://www.brasildefato.com.br/2024/03/01/bolsonarismo-busca-sobrevida-politica-ao-reforcar-o-populismo-religioso"

# Fazendo a requisição HTTP
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (código 200)
if response.status_code == 200:
    # Criando um objeto BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrando o elemento que contém o texto da notícia
    text_element = soup.find('div', class_='text-content')

    # Extraindo o texto se o elemento for encontrado
    if text_element:
        # Encontrando todos os parágrafos dentro do elemento
        paragraphs = text_element.find_all('p')

        # Iterando pelos parágrafos e imprimindo o texto
        for paragraph in paragraphs:
            print(paragraph.get_text())

    else:
        print("Elemento de texto não encontrado.")
else:
    print("Erro na requisição. Código de status:", response.status_code)


A manifestação em defesa do ex-presidente Jair Bolsonaro (PL), no último domingo(25), em São Paulo, reforçou a tática política adotada pela extrema direita no Brasil do chamado populismo religioso e demonstrou que o bolsonarismo busca sobrevida eleitoral alimentando a base cristã. O ato serviu para manter os apoiadores ligados ao conservadorismo nacional coesos e apontar o tom da disputa política nas eleições municipais. 
A manifestação, convocada logo após o cerco da Polícia Federal ao ex-presidente, contou com a presença de quatro governadores, deputados e prefeitos, além dos pastores Magno Malta (que também é senador pelo Espírito Santo) e Silas Malafaia, este último apontado como um dos financiadores.

Acuado pelas investigações da PF e pela inelegibilidade imposta pelo Tribunal Superior Eleitoral (TSE), Bolsonaro saiu em sua própria defesa, tentou suavizar o seu comportamento anterior e sugeriu passar uma borracha no passado.
No lugar dos ataques aos ministros do Supremo Tribunal 

In [99]:
import nltk
from nltk.tokenize import sent_tokenize

In [96]:
# Baixar os recursos necessários do NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [147]:
# Inicializando um conjunto de dados vazio
dataset_BdF = []

# Tokenizando em frases
for paragraph in paragraphs:
    sentences_BdF = sent_tokenize(paragraph.get_text())

    # Pegando apenas as 5 primeiras frases
    for idx, sentence_BdF in enumerate(sentences_BdF[:5], start=1):
        dataset_BdF.append(sentence_BdF)

    # Verificando se já pegou 5 frases e interrompendo o loop
    if len(dataset_BdF) >= 5:
        break

# Visualizando o conjunto de dados
for sentence in dataset_BdF:
    print(sentence)

A manifestação em defesa do ex-presidente Jair Bolsonaro (PL), no último domingo(25), em São Paulo, reforçou a tática política adotada pela extrema direita no Brasil do chamado populismo religioso e demonstrou que o bolsonarismo busca sobrevida eleitoral alimentando a base cristã.
O ato serviu para manter os apoiadores ligados ao conservadorismo nacional coesos e apontar o tom da disputa política nas eleições municipais.
A manifestação, convocada logo após o cerco da Polícia Federal ao ex-presidente, contou com a presença de quatro governadores, deputados e prefeitos, além dos pastores Magno Malta (que também é senador pelo Espírito Santo) e Silas Malafaia, este último apontado como um dos financiadores.
Acuado pelas investigações da PF e pela inelegibilidade imposta pelo Tribunal Superior Eleitoral (TSE), Bolsonaro saiu em sua própria defesa, tentou suavizar o seu comportamento anterior e sugeriu passar uma borracha no passado.
No lugar dos ataques aos ministros do Supremo Tribunal Fe

In [148]:
# Criar um novo conjunto de dados com palavras mascaradas
masked_BdF = [mask_words(sentence) for sentence in dataset_BdF]

# Visualizando o conjunto de dados mascarado
for sentence_BdF in masked_BdF:
    print(sentence_BdF)

A manifestação em defesa do ex-presidente [MASK] Bolsonaro (PL), no último domingo(25), em São Paulo, reforçou a tática política [MASK] pela extrema direita no Brasil do [MASK] populismo religioso e demonstrou que o [MASK] busca sobrevida eleitoral [MASK] [MASK] base cristã.
[MASK] ato serviu para manter [MASK] apoiadores ligados [MASK] conservadorismo nacional coesos e apontar o tom da disputa política nas eleições municipais.
A manifestação, convocada logo após o cerco da Polícia Federal [MASK] ex-presidente, contou com [MASK] presença [MASK] quatro governadores, deputados e [MASK] além dos pastores Magno Malta (que também é senador pelo Espírito [MASK] e Silas Malafaia, este último apontado como [MASK] dos financiadores.
Acuado pelas investigações da PF e pela [MASK] imposta pelo Tribunal Superior Eleitoral (TSE), Bolsonaro saiu em sua própria [MASK] tentou suavizar o seu comportamento anterior e [MASK] passar uma [MASK] no passado.
No [MASK] dos ataques aos ministros do [MASK] Trib

In [149]:
# Exemplo de sentenças
sentences = masked_BdF

# Tokenize as sentenças e substitua uma palavra por [MASK]
tokenized_sentences_BdF = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
print("Sentenças tokenizadas:", tokenized_sentences_BdF)

Sentenças tokenizadas: {'input_ids': tensor([[  101,   177, 11926,   173,  3573,   171,   294,   118,  1640,   103,
         20354,   716,   157,   113,   212, 22327,   114,   117,   202,  1957,
         11388,   113,  1906,   114,   117,   173,   710,  1033,   117, 14776,
         22288,   123, 19545,  1837,   103,   412,  4493,  5065,   202,   771,
           171,   103, 12936,   714,  7255,   122, 12219,   179,   146,   103,
          3344,  3051,   328,  8629,   103,   103,  1037,  7216,   119,   102],
        [  101,   103,  5291,  5083,   221,  3247,   103, 11781,   207,  8642,
           103, 14992,   714,  1772, 14336,  1409,   122, 12110, 22282,   146,
          5902,   180,  3623,  1837,   529,  3496,  9057,   119,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  10

In [150]:
# Acesse os IDs dos tokens
input_ids_BdF = tokenized_sentences_BdF["input_ids"]
print(input_ids)

tensor([[ 101,  510,  103,  453, 3155, 8573,  387,  358,  119,  102],
        [ 101,  103,  698,  202, 1997,  180,  651,  119,  102,    0]])


In [151]:
attention_mask_BdF = tokenized_sentences_BdF['attention_mask']
print(attention_mask_BdF)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [152]:
labels_BdF = input_ids_BdF.clone()
print(labels_BdF)

tensor([[  101,   177, 11926,   173,  3573,   171,   294,   118,  1640,   103,
         20354,   716,   157,   113,   212, 22327,   114,   117,   202,  1957,
         11388,   113,  1906,   114,   117,   173,   710,  1033,   117, 14776,
         22288,   123, 19545,  1837,   103,   412,  4493,  5065,   202,   771,
           171,   103, 12936,   714,  7255,   122, 12219,   179,   146,   103,
          3344,  3051,   328,  8629,   103,   103,  1037,  7216,   119,   102],
        [  101,   103,  5291,  5083,   221,  3247,   103, 11781,   207,  8642,
           103, 14992,   714,  1772, 14336,  1409,   122, 12110, 22282,   146,
          5902,   180,  3623,  1837,   529,  3496,  9057,   119,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,   177, 11926,   117,  4813,  2382,

In [153]:
# Configurar DataLoader
dataset2 = TensorDataset(input_ids_BdF, attention_mask_BdF, labels_BdF)
train_size = int(0.8 * len(dataset2))
val_size = len(dataset2) - train_size
train_dataset, val_dataset = random_split(dataset2, [train_size, val_size])

train_dataloader_BdF = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [155]:
train_model(model, train_dataloader_BdF, optimizer, scheduler, 10)

Epoch 1/10, Average Loss: 4.095005393028259
Epoch 2/10, Average Loss: 3.903255581855774
Epoch 3/10, Average Loss: 3.9597092866897583
Epoch 4/10, Average Loss: 3.837762951850891
Epoch 5/10, Average Loss: 3.8998477458953857
Epoch 6/10, Average Loss: 4.122880697250366
Epoch 7/10, Average Loss: 3.892215847969055
Epoch 8/10, Average Loss: 3.917101740837097
Epoch 9/10, Average Loss: 3.9680938720703125
Epoch 10/10, Average Loss: 3.8386093378067017
